In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold
import statistics
from shapely.geometry import Point,Polygon
from scipy.spatial.distance import cdist
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import MiniBatchKMeans
import shapely.wkt
import shapely
from matplotlib import pyplot

In [2]:
train = pd.read_csv('../data/raw_data/Train.csv')
submission = pd.read_csv('../data/raw_data/SampleSubmission.csv')
areas = pd.read_csv('../data/raw_data/protected_areas.csv')

In [3]:
train.columns

Index(['X', 'Y', 'target_2015', 'elevation', 'precip 2014-11-16 - 2014-11-23',
       'precip 2014-11-23 - 2014-11-30', 'precip 2014-11-30 - 2014-12-07',
       'precip 2014-12-07 - 2014-12-14', 'precip 2014-12-14 - 2014-12-21',
       'precip 2014-12-21 - 2014-12-28', 'precip 2014-12-28 - 2015-01-04',
       'precip 2015-01-04 - 2015-01-11', 'precip 2015-01-11 - 2015-01-18',
       'precip 2015-01-18 - 2015-01-25', 'precip 2015-01-25 - 2015-02-01',
       'precip 2015-02-01 - 2015-02-08', 'precip 2015-02-08 - 2015-02-15',
       'precip 2015-02-15 - 2015-02-22', 'precip 2015-02-22 - 2015-03-01',
       'precip 2015-03-01 - 2015-03-08', 'precip 2015-03-08 - 2015-03-15',
       'precip 2019-01-20 - 2019-01-27', 'precip 2019-01-27 - 2019-02-03',
       'precip 2019-02-03 - 2019-02-10', 'precip 2019-02-10 - 2019-02-17',
       'precip 2019-02-17 - 2019-02-24', 'precip 2019-02-24 - 2019-03-03',
       'precip 2019-03-03 - 2019-03-10', 'precip 2019-03-10 - 2019-03-17',
       'precip 2019-0

In [4]:
for area_index in range(len(areas)):
    poly = []
    for multipoly in shapely.wkt.loads(str(areas.iloc[area_index]['the_geom'])):
      multi_poly_point = multipoly.centroid
      poly.append((multi_poly_point.xy[0],multi_poly_point.xy[1]))
    point = Point(poly[0][0][0],poly[0][1][0])
    areas.loc[areas.index==area_index,'X'] = point.x
    areas.loc[areas.index==area_index,'Y'] = point.y

In [5]:
areas = areas.groupby('DISTRICT').mean()[['HOUSEHOLDS','MALE','FEMALE',"AREA","X","Y"]].reset_index()

In [6]:
train_points = train[['X','Y']].apply(lambda x:(x['X'],x['Y']),1).tolist()
areas_points = areas[['X','Y']].apply(lambda x:(x['X'],x['Y']),1).tolist()

In [7]:
areas.drop(['X','Y'],axis=1,inplace=True)

In [8]:
train['areasindex'] = np.argmin(cdist(train_points,areas_points),1)
train = train.merge(areas,how="left",left_on="areasindex",right_on=areas.index)

In [9]:
train.drop(['areasindex'],axis=1,inplace=True)

In [10]:
# train = train.merge(drought_risk,how="left",on=['X','Y'])

In [11]:
def rmse(x,y):
  return np.sqrt(mean_squared_error(x,y))
  

In [12]:
train.HOUSEHOLDS.fillna(train.HOUSEHOLDS.mean(),inplace=True)
train.MALE.fillna(train.MALE.mean(),inplace=True)
train.FEMALE.fillna(train.FEMALE.mean(),inplace=True)
train.AREA.fillna(train.AREA.mean(),inplace=True)

In [13]:
train_mean = train.groupby('LC_Type1_mode').mean()
train_std  = train.groupby('LC_Type1_mode').std()
train_mean_columns = list(train_mean.columns)
train_std_columns = list(train_std.columns)
for i in range(len(train_mean_columns)):
    train_mean_columns[i] = train_mean_columns[i]+'MEAN'
train_mean.columns = train_mean_columns
train = train.merge(train_mean,how="left",on="LC_Type1_mode")


for i in range(len(train_std_columns)):
    train_std_columns[i] = train_std_columns[i]+'STD'
train_std.columns = train_std_columns
train = train.merge(train_std,how="left",on="LC_Type1_mode")




In [14]:
train.columns

Index(['X', 'Y', 'target_2015', 'elevation', 'precip 2014-11-16 - 2014-11-23',
       'precip 2014-11-23 - 2014-11-30', 'precip 2014-11-30 - 2014-12-07',
       'precip 2014-12-07 - 2014-12-14', 'precip 2014-12-14 - 2014-12-21',
       'precip 2014-12-21 - 2014-12-28',
       ...
       'precip 2019-04-07 - 2019-04-14STD',
       'precip 2019-04-14 - 2019-04-21STD',
       'precip 2019-04-21 - 2019-04-28STD',
       'precip 2019-04-28 - 2019-05-05STD',
       'precip 2019-05-05 - 2019-05-12STD',
       'precip 2019-05-12 - 2019-05-19STD', 'HOUSEHOLDSSTD', 'MALESTD',
       'FEMALESTD', 'AREASTD'],
      dtype='object', length=129)

In [15]:
coordinates = np.vstack((train[['Y','X']].values))
indexes = np.arange(start=0, stop=len(coordinates), step=1)
mbkmeans = MiniBatchKMeans(n_clusters=10, batch_size=500,random_state=42).fit(coordinates[indexes])
train.loc[:, 'ClusterID'] = mbkmeans.predict(train[['Y', 'X']])

In [16]:
for i in ['DISTRICT']:
    
    train[i] = train[i].astype('category')

In [17]:
target = train.target_2015
train_restricted = train.iloc[:12000].copy()
target_restricted = target.iloc[:12000].copy()
train.drop(['target_2015'],axis=1,inplace=True)
train.drop(['Square_ID',"LC_Type1_mode"],axis=1,inplace=True)

In [18]:
train_rest= train.iloc[12000:]
target_rest = target.iloc[12000:]
train_rest=train_rest[target_rest<0.1]
target_rest=target_rest[target_rest<0.1]


In [19]:
# train_pred=pd.read_csv("../outputs/oof/train/"+"lgbm_0.0728_Kfold")
# train_pred["ame"]=(train_pred.target_2015-train_pred.target).abs()
# id_to_remove=train_pred[(train_pred.target_2015>0.9)&(train_pred.ame<0.04)].Square_ID
# len(id_to_remove)

166

In [20]:
train_restricted=pd.concat([train_restricted,train_rest])
target_restricted=pd.concat([target_restricted,target_rest])
train_restricted.target_2015=target_restricted.values
train_restricted=train_restricted[~train_restricted.Square_ID.isin(id_to_remove)]
target_restricted=train_restricted.target_2015
target_restricted.isna().sum()

0

In [21]:
train_restricted.drop(['target_2015'],axis=1,inplace=True)
train_restricted.drop(['Square_ID',"LC_Type1_mode"],axis=1,inplace=True)


In [33]:
Lightgbm = lgb.LGBMRegressor(n_estimators=100_000,random_state=1994,max_depth=8,learning_rate=0.02)
Kfolds = KFold(n_splits=10,shuffle=False)
test_prediction = np.zeros(len(train))
cv_score = []
for (train_index,test_index) in Kfolds.split(train_restricted,target_restricted):
  X_train,X_test = train_restricted.iloc[train_index],train_restricted.iloc[test_index]
  y_train,y_test = target_restricted.iloc[train_index],target_restricted.iloc[test_index]
  Lightgbm.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=50,eval_metric='rmse',verbose=-1)  


  test_pred = Lightgbm.predict(train)
  validation_pred= Lightgbm.predict(X_test)


  cv_score.append(rmse(validation_pred,y_test))

 
  test_prediction+=test_pred
    
    
print("CV score is ={}".format(np.mean(cv_score)))

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[181]	valid_0's rmse: 0.0863084	valid_0's l2: 0.00744913
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[116]	valid_0's rmse: 0.0527329	valid_0's l2: 0.00278076
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[85]	valid_0's rmse: 0.0832981	valid_0's l2: 0.00693857
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[84]	valid_0's rmse: 0.154911	valid_0's l2: 0.0239973
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[228]	valid_0's rmse: 0.182613	valid_0's l2: 0.0333477
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[58]	valid_0's rmse: 0.124025	valid_0's l2: 0.0153822
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration

In [23]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

# # sorted(zip(clf.feature_importances_, X.columns), reverse=True)
# feature_imp = pd.DataFrame(sorted(zip(Lightgbm.feature_importances_,train.columns),reverse=True), columns=['Value','Feature'])

# plt.figure(figsize=(30, 20))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.show()

In [24]:
np.histogram(target)

(array([14387,   328,   265,   183,   149,   136,   117,   125,   134,
          642]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

In [111]:
kfold=pd.read_csv("xgbosot_kfold")

In [112]:
# test_pred_clipped=np.maximum(test_prediction/10,0)
# test_pred_clipped=np.minimum(test_pred,1)
submission['target_2019'] = ((np.absolute(test_prediction/10)))*0.6+kfold.target_2019*0.4
# submission['target_2019'] = test_pred_clipped


In [113]:
submission['target_2019'] = submission['target_2019']*0.94

# ((np.absolute(test_prediction/10)))*0.8
submission.to_csv('lgbm_0.074_92.csv',index=False)

In [114]:
submission.loc[submission['target_2019']<=0.009,"target_2019"]=0.000

In [115]:
submission.to_csv('lgbm_94_9.csv',index=False)

In [116]:
submission.target_2019.value_counts()

0.000000    5513
0.014511     255
0.015056      96
0.015080      66
0.014497      57
            ... 
0.025590       1
0.048392       1
0.052224       1
0.009634       1
0.221865       1
Name: target_2019, Length: 7941, dtype: int64

In [117]:
submission['target_2015']=target
submission[submission["target_2015"]==0].target_2019.sum()


168.80938882138202

In [118]:
submission.loc[submission["target_2015"]==0,"target_2019"]*=0.5  # try 0.4 
print(submission[submission["target_2015"]==0].target_2019.sum())
del submission['target_2015']


84.40469441069101


In [119]:
submission.to_csv('lgbm_94_9_2.csv',index=False)